In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['HF_HOME'] = '/shared/data3/pk36/.cache'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
!export HF_HOME=/shared/data3/pk36/.cache
os.environ['CUDA_VISIBLE_DEVICES'] = "6,7"

In [ ]:
from debate import DebateNode
from paper_details import Paper
from persona import PaperAuthor
from moderator import Moderator
import argparse
from typing import List
from vllm import LLM
import os
import json
from data_pairer import parse_papers
import numpy as np

In [12]:
class Args():
    def __init__(self):
        self.log_dir = "logs"
        self.focus_paper = "https://arxiv.org/pdf/2406.11709" # treeinstruct
        self.cited_paper = "https://arxiv.org/pdf/2310.10648" # bridge
        self.topic = {'argument_title': "helping students fix their errors", 'description': ""}

args = Args()

In [8]:
if not os.path.exists(args.log_dir):
    os.makedirs(args.log_dir)

# if not os.path.exists("data.json"):
parse_papers(args.focus_paper, args.cited_paper)
with open('data.json', 'r') as file:
    data = json.load(file)

In [9]:
model_server = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct",tensor_parallel_size=2, gpu_memory_utilization=0.5, max_num_seqs=100) #,enable_prefix_caching=True)

INFO 11-29 12:05:23 config.py:905] Defaulting to use mp for distributed inference
WARNING 11-29 12:05:23 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-29 12:05:23 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-29 12:05:23 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


(VllmWorkerProcess pid=3743417) INFO 11-29 12:05:31 model_runner.py:1067] Loading model weights took 7.5122 GB
INFO 11-29 12:05:31 model_runner.py:1067] Loading model weights took 7.5122 GB
INFO 11-29 12:05:32 distributed_gpu_executor.py:57] # GPU blocks: 15590, # CPU blocks: 4096
INFO 11-29 12:05:32 distributed_gpu_executor.py:61] Maximum concurrency for 131072 tokens per request: 1.90x
INFO 11-29 12:05:34 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-29 12:05:34 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=3743417) INFO 11-29 12:05:34 model_runner.py:1395] Capturing the mo

In [10]:
item = data[0]
f_pap = item['focus']
c_pap = item['cited']

focus_paper = PaperAuthor(
    model = model_server,
    paper = Paper(f_pap),
    focus=True,
    id=0
)

cited_paper = PaperAuthor(
    model = model_server,
    paper = Paper(c_pap),
    focus=False,
    id=1
)

100%|██████████| 3/3 [00:01<00:00,  2.84it/s]


In [13]:
moderator = Moderator(model_server)

paper_authors = [focus_paper, cited_paper]
leaf_node_label = args.topic

if args.log_dir != "":
    with open(os.path.join(args.log_dir, 'self_deliberation.txt'), 'w') as f:
        f.write(f'Topic: {args.topic}\n\n')

# each node has a topic
root_node = DebateNode(leaf_node_label)
subtrees = root_node.conduct_self_deliberation(leaf_node_label, paper_authors, log=args.log_dir) # k new, finer topics to discuss

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.87s/it, est. speed input: 165.58 toks/s, output: 62.40 toks/s]


OSError: [Errno 28] No space left on device

In [14]:
root_node.self_delib

{0: []}

In [10]:
root_node.preemption

{0: {'Does my paper also address the claim, "bidirectional language model architecture"?': ['We used a beam size of 21 and a = 0.3 for both WSJ only and the semi-supervised setting. Our results in Table 4 show that despite the lack of task-specific tuning our model performs surprisingly well, yielding better results than all previously reported models with the exception of the Recurrent Neural Network Grammar [8]',
   'Introduction Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [38, 24, 15]'],
  'Does my paper also address the claim, "advancements in pre-training objectives"?': ['During inference, we Table 4: The Transformer generalizes

In [11]:
root_node.children

[Transformer Architecture Offers Superior Performance,
 Transformer Architecture Overcomes Recurrent Model Limitations]

In [12]:
convo_history = root_node.children[0].conduct_debate(paper_authors)


PRESENT ARGUMENT FOR AUTHOR 0:



Compiling FSM index for all state transitions: 100%|██████████| 57/57 [00:02<00:00, 25.37it/s]


You are an author of a paper that is debating another author on your claim topic:
	- Topic: Transformer Architecture Offers Superior Performance
	- Topic Description: Our paper proposes a novel Transformer architecture that outperforms existing models in machine translation tasks. This is supported by evidence that our model achieves superior quality while requiring significantly less training time, as shown in Retrieved Evidence #1. Furthermore, our model generalizes well to other tasks such as English constituency parsing, as demonstrated in Retrieved Evidence #3.

Your debate claim is that your paper's contributions towards the topic are all novel relative to the other paper. Refer to their arguments and presented evidence, as well as your own paper's segments as evidence when refining your arguments.

You used the following evidence to support your arguments:
	- Evidence #1. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispens

In [13]:
print(convo_history)

Debate Topic Information:
	- Topic: Transformer Architecture Offers Superior Performance
	- Topic Description: Our paper proposes a novel Transformer architecture that outperforms existing models in machine translation tasks. This is supported by evidence that our model achieves superior quality while requiring significantly less training time, as shown in Retrieved Evidence #1. Furthermore, our model generalizes well to other tasks such as English constituency parsing, as demonstrated in Retrieved Evidence #3.

Debate History:

	-Author 0: I argue that novel architecture and training methods outperform existing models. Our paper introduces a novel Transformer architecture that outperforms existing models in machine translation tasks. This is supported by evidence that our model achieves superior quality while requiring significantly less training time, as shown in Retrieved Evidence #1. Furthermore, our model generalizes well to other tasks such as English constituency parsing, as dem

In [15]:
root_node.children[0].response[0]

{'argument_list': [{'title': 'Argument 1',
   'description': 'Argument 1 description'},
  {'title': 'Argument 2', 'description': 'Argument 2 description'}]}

In [ ]:
conversation_history = []

queue_of_rounds: List[DebateNode] = []
queue_of_rounds.extend(subtrees)


while len(queue_of_rounds) > 0:
    round = queue_of_rounds.pop(0)
    conversation, new_focus_arg, new_cited_arg = round.conduct_debate(focus_paper, cited_paper)
    conversation_history.extend(conversation)
    if moderator.is_expand(round.self_delib, [new_focus_arg, new_cited_arg]):
        new_subtrees = round.conduct_self_deliberation(round.round_topic, paper_authors)
        queue_of_rounds.extend(new_subtrees)

with open('conversation_history.txt', 'w+') as f:
    f.write('\n'.join(conversation_history))